In [1]:
from math import atan2, pi, fmod, ceil

from rlbot.utils.game_state_util import Rotator, Vector3, Physics
from rlbot.agents.base_agent import SimpleControllerState

from rlutilities.linear_algebra import (
    vec3 as rlu_vec3,
    euler_to_rotation,
    mat3,
    mat4,
    dot,
    norm,
    vec2 as rlu_vec2,
    vec4 as rlu_vec4,
    transpose,
    angle_between,
    axis_to_rotation,
    normalize,
    cross,
)
from rlutilities.simulation import Game, Field, obb

from util.vec import Vec3
from util.orientation import Orientation


In [2]:
def to_rlu_vec(v) -> rlu_vec3:
    return rlu_vec3(v.x, v.y, v.z)


def rlu_to_Vec3(v) -> Vec3:
    return Vec3(v[0], v[1], v[2])

def rotate_vector(v: Vec3, r: mat3, around: Vec3=None) -> Vec3:
    out = dot(r,to_rlu_vec(v))
    return Vec3(out[0], out[1], out[2])

def rotate_vector(v: Vec3, r: mat3, around: Vec3=None) -> Vec3:
    out = dot(r,to_rlu_vec(v))
    return Vec3(out[0], out[1], out[2])



In [5]:
rotate_vector(Vec3(1, 2, 3), Orientation(Rotator(0, 0, 0)).to_rot_mat())

Vec3(1.00, 2.00, 3.00)

In [6]:
rotate_vector(Vec3(1, 2, 3), Orientation(Rotator(0, pi/2, 0)).to_rot_mat())

Vec3(-2.00, 1.00, 3.00)

In [22]:
def rot_unrot_(x, y, z, rot):
    v = Vec3(x, y, z)
    o = Orientation(rot)
    rot_mat = o.to_rot_mat()

    rotated = rotate_vector(v, rot_mat)
    unrotated = rotate_vector(rotated, transpose(rot_mat))
    
    print(f"{v} {unrotated}. Inbetween: {rotated}")
def rot_unrot(x, y, z, yaw): 
    rot_unrot_(x,y,z, Rotator(0, yaw, 0))

In [23]:
rot_unrot(1, 2, 3, 0)
rot_unrot(1, 2, 3, pi/2)

Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(1.00, 2.00, 3.00)
Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(-2.00, 1.00, 3.00)


In [28]:
rot_unrot_(1, 2, 3, Rotator(pi, 0, 0))
rot_unrot_(1, 2, 3, Rotator(pi, pi/3, 0))
rot_unrot_(1, 2, 3, Rotator(pi, 0, -pi/6))
rot_unrot_(1, 2, 3, Rotator(pi*3/2, -pi/2, pi/3))


Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(-1.00, 2.00, -3.00)
Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(-2.23, 0.13, -3.00)
Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(-1.00, 0.23, -3.60)
Vec3(1.00, 2.00, 3.00) Vec3(1.00, 2.00, 3.00). Inbetween: Vec3(3.60, 0.23, -1.00)


In [35]:
# Lets test adding rotation
def add_rots(orig, second):
    orig_rot_mat = Orientation(orig).to_rot_mat()
    rot_mat_upd: mat3 = Orientation(second).to_rot_mat()

    rot_mat = dot(rot_mat_upd, orig_rot_mat)
    rot = rot_mat_to_rot(rot_mat)

    print(f"{rot.pitch}, {rot.yaw}, {rot.roll}")

def rot_mat_to_rot(theta: mat3) -> Rotator:
    pitch = atan2(theta[(2, 0)], norm(rlu_vec2(theta[(0, 0)], theta[(1, 0)])))
    yaw = atan2(theta[(1, 0)], theta[(0, 0)])
    roll = atan2(-theta[(2, 1)], theta[(2, 2)])
    return Rotator(pitch, yaw, roll)


In [38]:
add_rots(Rotator(0, 0.5, 0), Rotator(0, 0.2, 0))
add_rots(Rotator(0, 0.5, 0), Rotator(0, -0.3, 0))

0.0, 0.7000000163252541, -0.0
0.0, 0.20000000922679514, -0.0


In [39]:
add_rots(Rotator(1, 0.5, 0), Rotator(0, 0.2, 0))

0.9999999593317271, 0.7000000159777006, -0.0


In [40]:
add_rots(Rotator(1, 0.5, 0), Rotator(0, 0, 0))

0.9999999593317271, 0.499999985496137, -0.0


In [8]:
z

Vec3(-3.00, 2.00, 1.00)

# Goal: Rotate to match "up"

We want to take an orientation and rotate it so that it's
up matches the normal. This may mean adjusting pitch
or roll but we want to not lock the yaw

In [65]:
def rot_mat_to_rot(theta: mat3) -> Rotator:
    pitch = atan2(theta[(2, 0)], norm(rlu_vec2(theta[(0, 0)], theta[(1, 0)])))
    yaw = atan2(theta[(1, 0)], theta[(0, 0)])
    roll = atan2(-theta[(2, 1)], theta[(2, 2)])
    return Rotator(pitch, yaw, roll)

def printO(o: Orientation):
    print(f"f: {o.forward}, r: {o.right}, u: {o.up}")

In [14]:
# Case where normal and up match already
o = Orientation(Rotator(0, pi/2, 0))
n = Vec3(0, -0.05, 0.9).normalized()


angle_between(to_rlu_vec(o.up), to_rlu_vec(n))

0.05549851804971695

In [15]:
print(o.to_rot_mat())
print(euler_to_rotation(rlu_vec3(0, pi/2, 0)))
print(dot(o.to_rot_mat(), transpose(o.to_rot_mat())))

0.000000 -1.000000 -0.000000
1.000000 0.000000 0.000000
0.000000 -0.000000 1.000000

-0.000000 -1.000000 0.000000
1.000000 -0.000000 -0.000000
0.000000 -0.000000 1.000000

1.000000 0.000000 0.000000
0.000000 1.000000 0.000000
0.000000 0.000000 1.000000



In [68]:
orig = o.to_rot_mat()
r = rot_mat_to_rot(mat3(
    orig[0, 0], orig[0, 1], n.x,
    orig[1, 0], orig[1, 1], n.y,
    orig[2, 0], orig[2, 1], n.z
))
printO(o)
printO(Orientation(r))
(r.pitch, r.yaw, r.roll)
# didn't work

f: Vec3(0.00, 0.00, 1.00), r: Vec3(1.00, 0.00, -0.00), u: Vec3(-0.00, 1.00, 0.00)
f: Vec3(0.00, 0.00, 1.00), r: Vec3(0.00, 1.00, -0.00), u: Vec3(-1.00, 0.00, 0.00)


(1.5707963267948966, 0.0, 6.123234262925839e-17)

In [66]:
def rotate_by_axis(o, n):
    normal = n
    orientation = o
    angle = angle_between(to_rlu_vec(normal), to_rlu_vec(orientation.up))
    rotate_axis = cross(to_rlu_vec(normal), to_rlu_vec(orientation.up))
    ortho = (
        normalize(rotate_axis) * -angle
    )
    rot_mat_initial: mat3 = orientation.to_rot_mat()
    rot_mat_adj = axis_to_rotation(ortho)

    rot_mat = dot(rot_mat_adj, rot_mat_initial )
    r = rot_mat_to_rot(rot_mat)
    printO(o)
    printO(Orientation(r))
    return r

r = rotate_by_axis(o, n)
(r.pitch, r.yaw, r.roll)


f: Vec3(0.00, 0.00, 1.00), r: Vec3(1.00, 0.00, -0.00), u: Vec3(-0.00, 1.00, 0.00)
f: Vec3(0.00, -1.00, -0.00), r: Vec3(1.00, 0.00, 0.00), u: Vec3(-0.00, -0.00, 1.00)


(-4.37113882867379e-08, -1.5707963267948966, -2.6765506200124328e-24)

In [69]:
r = rotate_by_axis(
    Orientation(Rotator(pi/2, pi/2, pi/2)), # on the wall, facing up
    Vec3(0, 0, 1)
)
(r.pitch, r.yaw, r.roll)

f: Vec3(0.00, 0.00, 1.00), r: Vec3(0.00, 1.00, -0.00), u: Vec3(-1.00, 0.00, 0.00)
f: Vec3(1.00, 0.00, -0.00), r: Vec3(-0.00, 1.00, 0.00), u: Vec3(0.00, -0.00, 1.00)


(-4.37113882867379e-08, 6.123233601181349e-17, -2.6765506200124328e-24)

In [70]:
o = Orientation(Rotator(pi/2, 0, pi/2)) # on the wall, facing up
r = rotate_by_axis(
    o,
    Vec3(0, 0, 1)
)
(r.pitch, r.yaw, r.roll)

f: Vec3(0.00, 0.00, 1.00), r: Vec3(1.00, 0.00, -0.00), u: Vec3(-0.00, 1.00, 0.00)
f: Vec3(0.00, -1.00, -0.00), r: Vec3(1.00, 0.00, 0.00), u: Vec3(-0.00, -0.00, 1.00)


(-4.37113882867379e-08, -1.5707963267948966, -2.6765506200124328e-24)

In [63]:
o = Orientation(Rotator(pi/2, 0, pi/2)) # on the wall, facing up
print(o.to_rot_mat())
n = Vec3(0, 0, 1)


0.000000 1.000000 -0.000000
0.000000 0.000000 1.000000
1.000000 -0.000000 0.000000



In [56]:
o = Orientation(Rotator(pi/2, pi/2, pi/2))
print(o.forward)
o = Orientation(Rotator(0, 0, 0))
print(o.forward)

Vec3(0.00, 0.00, 1.00)
Vec3(1.00, 0.00, 0.00)


# Goal: Apply rotation 

In [74]:
base_r = Rotator(pi/2, pi/2, pi/2)
base_o = Orientation(base_r)
printO(base_o)

upd_r = Rotator(0, -pi, 0)
upd_o = Orientation(upd_r)
printO(upd_o)

combined = dot(upd_o.to_rot_mat(), base_o.to_rot_mat())
printO(Orientation.from_rot_mat(combined))

f: Vec3(0.00, 0.00, 1.00), r: Vec3(0.00, 1.00, -0.00), u: Vec3(-1.00, 0.00, 0.00)
f: Vec3(-1.00, -0.00, 0.00), r: Vec3(0.00, -1.00, -0.00), u: Vec3(0.00, 0.00, 1.00)
f: Vec3(0.00, -0.00, 1.00), r: Vec3(0.00, -1.00, -0.00), u: Vec3(1.00, 0.00, 0.00)


In [76]:
# we want f: Vec3(0, 0, -1)
combined = dot(base_o.to_rot_mat(), upd_o.to_rot_mat())
printO(Orientation.from_rot_mat(combined))

f: Vec3(0.00, -0.00, -1.00), r: Vec3(0.00, -1.00, 0.00), u: Vec3(-1.00, -0.00, 0.00)


In [3]:
x = Vec3(0, 1, 2)
print(*x)
y = Vector3(*x)
print(y.y)
p = Physics()
print(p.location)

0.0 1.0 2.0
1.0
None
